In [ ]:
!pip install tensorboard --quiet
%reload_ext tensorboard

In [ ]:
# Tensorflow Modules
import tensorflow as tf
from tensorboard.plugins.hparams import api as hp
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Fundamental Modules
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from itertools import product
import pandas as pd
import numpy as np
import random
import datetime
import zipfile
import pathlib
import os

In [ ]:
def view_random_images(
    target_dir,
    target_class
    ):
  target_folder = os.path.join(target_dir, target_class)
  random_image = random.sample(os.listdir(target_folder), 1)
  img = mpimg.imread(target_folder + "/" + random_image[0])

  plt.imshow(img)
  plt.title(target_class + " with a shape of: " + str(img.shape))
  plt.axis("off")

  return img

def resize_images(
    file_path: str,
    img_shape: int = 224
    ):
  image = tf.io.read_file(file_path)
  image = tf.image.decode_image(image)
  image = tf.image.resize(image,
                          size=[img_shape, img_shape]
                          )
  image = tf.expand_dims(image / 255., axis=0)

  return image

def plot_predictions(
    target_dir: str,
    model
    ):
  class_labels = ["pizza", "steak"]
  target_class = random.choice(class_labels)
  target_folder = os.path.join(target_dir, target_class)

  random_image = random.choice(os.listdir(target_folder))
  image_path = os.path.join(target_folder,
                            random_image)
  resized_image = resize_images(image_path)

  prediction = model.predict(resized_image)
  prediction_object = class_labels[int(tf.round(prediction))]

  image = mpimg.imread(image_path)
  plt.imshow(image)
  plt.title(f"Predicted Object: {prediction_object}")
  plt.axis(False)
  plt.show()

In [ ]:
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/pizza_steak.zip

zip_ref = zipfile.ZipFile("pizza_steak.zip")
zip_ref.extractall()
zip_ref.close()

In [ ]:
data_dir = pathlib.Path("pizza_steak/train")
class_names = np.array(sorted([item.name for item in data_dir.glob("*")]))
print(class_names)

In [ ]:
class_type = ["steak", "pizza"]
num_rows = len(class_type)
num_cols = 5

fig, axes = plt.subplots(num_rows, num_cols, figsize=(16, 7))

for i, food in enumerate(class_type):
    for j in range(num_cols):
        ax = axes[i, j]
        img = view_random_images(target_dir="pizza_steak/train/",
                                 target_class=food)
        ax.imshow(img)
        ax.set_title(food + str(img.shape))
        ax.axis("off")

plt.tight_layout()
plt.show()

In [ ]:
tf.random.set_seed(1212124)

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
    )


train_data = train_datagen.flow_from_directory(
    directory="/content/pizza_steak/train",
    batch_size=32,
    target_size=(224,224),
    class_mode="binary",
    shuffle=False,
    seed=1212124
  )


shuffled_training_data = train_datagen.flow_from_directory(
    directory="/content/pizza_steak/train",
    batch_size=32,
    target_size=(224,224),
    class_mode="binary",
    shuffle=True,
    seed=1212124
  )

non_augmented_train_datagen = ImageDataGenerator(
    rescale=1./255
)

non_augmented_train_data = non_augmented_train_datagen.flow_from_directory(
    directory="/content/pizza_steak/train",
    batch_size=32,
    target_size=(224,224),
    class_mode="binary",
    shuffle=False,
    seed=1212124
  )


test_datagen = ImageDataGenerator(
    rescale=1./255
    )

test_data = test_datagen.flow_from_directory(
    directory="/content/pizza_steak/test",
    batch_size=32,
    target_size=(224,224),
    class_mode="binary",
    seed=1212124
  )

In [ ]:
non_augmented_images, non_augmented_labels = non_augmented_train_data.next()
augmented_images, augmented_labels = train_data.next()

In [ ]:
print(f"Non-augmented Image Shape {non_augmented_images.shape}, \nAugmented Image Shape: {augmented_images.shape}")

In [ ]:
random_number = random.randint(0, 32)
plt.imshow(non_augmented_images[random_number])
plt.title(f"Original Image")
plt.axis(False)
plt.figure()
plt.imshow(augmented_images[random_number])
plt.title(f"Augmented Image")
plt.axis(False);

In [ ]:
tf.random.set_seed(42)

# Visualize the training on Tensorboard
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

# Save model checkpoint with best epoch
cb_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath="model_checkpoint/model-{epoch:02d}-{accuracy:.3f}.hdf5",
    monitor="val_accuracy",
    mode="max",
    save_best_only=True,
    save_weights_only=True,
    verbose=0
)

# Drop the learning rate when model does not improve its performance metric
cb_reducelr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor="val_accuracy",
    mode="max",
    factor=0.1,
    patience=5,
    verbose=0,
    min_lr=1e-7
)

# Force stop the training if model does not improve for a specific number of epochs
cb_earlystop = tf.keras.callbacks.EarlyStopping(
    monitor="val_accuracy",
    mode="max",
    min_delta=0.001,
    patience=10,
    verbose=0,
)


In [ ]:
class FV_101(tf.keras.Model):
  def __init__(
        self,
        filters: int,
        kernel_size: int,
        strides: int,
        activations: str,
        num_inputs: tuple[int],
        pool_size: tuple[int],
        units: int
        ):

    super(FV_101, self).__init__(name="")
    self.filters=filters
    self.kernel_size=kernel_size
    self.strides=strides
    self.activations = activations
    self.num_inputs=num_inputs
    self.pool_size=pool_size
    self.units=units

    self.conv2d_1 = tf.keras.layers.Conv2D(
        filters=self.filters,
        kernel_size=self.kernel_size,
        strides=self.strides,
        activation=self.activations,
        kernel_initializer=tf.keras.initializers.LecunNormal(seed=1212124),
        bias_initializer=tf.keras.initializers.LecunNormal(seed=1212124),
        input_shape=self.num_inputs,
        name="conv2d_1"
    )

    self.maxpool_1 = tf.keras.layers.MaxPool2D(
        pool_size=self.pool_size,
        strides=self.strides,
        padding="valid",
        name="maxpool_1"
    )


    self.conv2d_2 = tf.keras.layers.Conv2D(
        filters=self.filters,
        kernel_size=self.kernel_size,
        strides=self.strides,
        activation=self.activations,
        kernel_initializer=tf.keras.initializers.LecunNormal(seed=1212124),
        bias_initializer=tf.keras.initializers.LecunNormal(seed=1212124),
        name="conv2d_2"
    )

    self.maxpool_2 = tf.keras.layers.MaxPool2D(
        pool_size=self.pool_size,
        strides=self.strides,
        padding="valid",
        name="maxpool_2"
    )

    self.conv2d_3 = tf.keras.layers.Conv2D(
        filters=self.filters,
        kernel_size=self.kernel_size,
        strides=self.strides,
        activation=self.activations,
        kernel_initializer=tf.keras.initializers.LecunNormal(seed=1212124),
        bias_initializer=tf.keras.initializers.LecunNormal(seed=1212124),
        name="conv2d_3"
    )

    self.maxpool_3 = tf.keras.layers.MaxPool2D(
        pool_size=self.pool_size,
        strides=self.strides,
        padding="valid",
        name="maxpool_3"
    )

    self.conv2d_4 = tf.keras.layers.Conv2D(
        filters=self.filters,
        kernel_size=self.kernel_size,
        strides=self.strides,
        activation=self.activations,
        kernel_initializer=tf.keras.initializers.LecunNormal(seed=1212124),
        bias_initializer=tf.keras.initializers.LecunNormal(seed=1212124),
        name="conv2d_4"
    )

    self.maxpool_4 = tf.keras.layers.MaxPool2D(
        pool_size=self.pool_size,
        strides=self.strides,
        padding="valid",
        name="maxpool_4"
    )

    self.flat = tf.keras.layers.Flatten(name="flat_1")
    self.dns = tf.keras.layers.Dense(
        units=self.units,
        activation=self.activations,
        kernel_initializer=tf.keras.initializers.LecunNormal(seed=1212124),
        bias_initializer=tf.keras.initializers.LecunNormal(seed=1212124),
        name="dense_1"
        )
    self.outpt = tf.keras.layers.Dense(1, activation="sigmoid", name="output")


  def call(self, inputs):
      x = self.conv2d_1(inputs)
      x = self.maxpool_1(x)
      x = self.conv2d_2(x)
      x = self.maxpool_2(x)
      x = self.conv2d_3(x)
      x = self.maxpool_3(x)
      x = self.conv2d_4(x)
      x = self.maxpool_4(x)
      x = self.flat(x)
      x = self.dns(x)
      x = self.outpt(x)

      return x

In [ ]:
cnn_model = FV_101(
    filters=10,
    kernel_size=3,
    strides=2,
    activations="relu",
    num_inputs=(224, 224, 3),
    pool_size=2,
    units=100
)

cnn_model.compile(
    loss="binary_crossentropy",
    optimizer=tf.keras.optimizers.Adam(),
    metrics=["accuracy"])

cnn_model.fit(
    shuffled_training_data,
    epochs=100,
    steps_per_epoch=len(train_data),
    batch_size=32,
    callbacks=[
        cb_checkpoint,
        cb_reducelr,
        cb_earlystop,
        tensorboard_callback
        ],
    verbose=1,
    validation_data=test_data,
    validation_steps=len(test_data)
    )

In [ ]:
cnn_model.summary(
    expand_nested=True,
    show_trainable=True
    )

In [ ]:
# Access to tensorboard
!tensorboard dev upload --logdir ./logs/ \
  --name "Food101 Image Classification Model Training"

In [ ]:
class_type = ["steak", "pizza"]
num_rows = len(class_type)
num_cols = 2

fig, axes = plt.subplots(num_rows, num_cols, figsize=(16, 7))

for i, food in enumerate(class_type):
    for j in range(num_cols):
        ax = axes[i, j]
        img = view_random_images(target_dir="/content/oos_image/",
                                 target_class=food)
        ax.imshow(img)
        ax.set_title(food + str(img.shape))
        ax.axis(False)

plt.tight_layout()
plt.show()

In [ ]:
class_names = ["pizza", "steak"]
base_dir = "/content/oos_image/"

image_list = []

for class_name in class_names:
    class_dir = os.path.join(base_dir, class_name)
    if os.path.exists(class_dir):
        image_files = os.listdir(class_dir)
        for image_file in image_files:
            image_path = os.path.join(class_dir, image_file)
            processed_image = resize_images(image_path)
            image_list.append(processed_image)
    else:
        print(f"Directory '{class_name}' not found.")

In [ ]:
best_cnn_model = FV_101(
    filters=10,
    kernel_size=3,
    strides=2,
    activations="relu",
    num_inputs=(224, 224, 3),
    pool_size=2,
    units=100
)
best_cnn_model(tf.zeros((1,224, 224, 3)))
best_cnn_model.load_weights("/content/model-30-0.843.hdf5")

In [ ]:
test_prediction = best_cnn_model.predict(image_list)
print(class_names[int(tf.round(test_prediction))])

In [ ]:
plot_predictions(
    target_dir = "/content/pizza_steak/test/",
    model=best_cnn_model)

In [ ]:
plot_predictions(
    target_dir = "/content/oos_image/",
    model=best_cnn_model)